Build the docker image and upload the model to GCP Artifact Registry 

In [10]:
# Step-1 : Build Docker Image 
! docker build -t ad-model-new .

# Step-2 : Tag Docker Image 
! docker tag ad-model-new gcr.io/{your-project-id}/ad-model-new

# Step-3 : Push docker image to Artifact Registry 
! docker push gcr.io/{your-project-id}/ad-model-new


[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for gcr.io/deeplearning-platform-release/skl  0.1s
[+] Building 0.5s (2/3)              

One time Setup - Create a New Service Account in GCP before executing the below gcloud commands    

In [ ]:

!gcloud projects add-iam-policy-binding {your-project-id} \
    --member=serviceAccount:vertex-ai-sa@{your-project-id}.iam.gserviceaccount.com \
    --role=roles/aiplatform.customCodeServiceAgent

!gcloud projects add-iam-policy-binding {your-project-id} \
    --member=serviceAccount:vertex-ai-sa@{your-project-id}.iam.gserviceaccount.com \
    --role=roles/aiplatform.admin

!gcloud projects add-iam-policy-binding {your-project-id} \
    --member=serviceAccount:vertex-ai-sa@{your-project-id}.iam.gserviceaccount.com \
    --role=roles/storage.objectAdmin

In [12]:
from google.cloud import aiplatform
aiplatform.init(project='{your-project-id}', location='us-east1')

In [2]:
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "{path to json credentials}"

Create a training job from the image from Container registry in Vertex AI 

In [14]:
display_name = 'adclick-model-training'

container_uri = 'gcr.io/{your-project-id}/ad-model'

bucket = 'vertex-ai-udemy'

container_deployment = aiplatform.CustomContainerTrainingJob(display_name=display_name,
                                               container_uri=container_uri,
                                               staging_bucket=bucket
                                                    )

model = container_deployment.run(replica_count=1,machine_type = "n1-standard-4",
                                accelerator_type= "ACCELERATOR_TYPE_UNSPECIFIED",
                                accelerator_count=0)


Training Output directory:
gs://vertex-ai-udemy/aiplatform-custom-training-2022-12-24-00:52:07.375 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2701920220326395904?project=470181514403
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8995159739855798272?project=470181514403
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/2701920220326395904 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/2701920220326395904 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/2701920220326395904 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/2701920220326395904 current state:
PipelineState.PIPEL

Upload the model artifact to Vertex AI Model Registry 

In [19]:
artifact_uri = "gs://your-bucket-name/adclick-model-artifact"

display_name = "ad-click-model"
ad_click_model = aiplatform.Model.upload(display_name=display_name,
                                  artifact_uri=artifact_uri,
                                  serving_container_image_uri=container_uri,is_default_version=True,
                                                version_aliases=["first stable version"],
                                                version_description="first stable version ")

Creating Model
Create Model backing LRO: projects/470181514403/locations/us-central1/models/5680202413416382464/operations/495835007747620864
Model created. Resource name: projects/470181514403/locations/us-central1/models/5680202413416382464@1
To use this Model in another session:
model = aiplatform.Model('projects/470181514403/locations/us-central1/models/5680202413416382464@1')
